# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Introdução à Otimização Logística_ <h6>Prof. Rafael Augusto de Melo</h6>

## Instruções
1. Para uso de GPU : _Runtime_ > _altere o runtime_ > _acelerador de Hardware_ = _GPU_.
2. Execute a célula abaixo ( clique e pressione Ctrl+Enter) para instalar Julia, IJulia e outros pacotes.
3. Recarregue esta página (Ctrl+R, ou ⌘+R, ou F5) e continue para próxima seção.

_Notas_:
* Caso o Colab Runtime resete (Ex: inatividade), repita o passo 2 e 3.
* Template adaptado por João Paulo Gomes Bernardino


In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.7" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Julia `julia -v` instalado com sucesso!"
  echo "Recarregue esta página (pressione Ctrl+R, ⌘+R ou a tecla F5) e depois"
  echo "vá para a seção 'Verificação da Instalação'."
fi

Installing Julia 1.6.7 on the current Colab Runtime...
2023-09-13 05:11:36 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.7-linux-x86_64.tar.gz [114281842/114281842] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Julia julia version 1.6.7 instalado com sucesso!
Recarregue esta página (pressione Ctrl+R, ⌘+R ou a tecla F5) e depois
vá para a seção 'Verificação da Instalação'.


# Precisa de ajuda?

* Apredizado: https://julialang.org/learning/
* Documentação: https://docs.julialang.org/
* Questões e Discurssões:
  * https://discourse.julialang.org/
  * http://julialang.slack.com/
  * https://stackoverflow.com/questions/tagged/julia

Adicione mais céluas utilizando  `+ Code`

Divirta-se!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />

In [3]:
versioninfo()

Julia Version 1.4.1
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-8.0.1 (ORCJIT, skylake)


Para instalar mais pacotes adicione
`;Pkg.add("Pacote")` na célula abaixo

In [4]:
import Pkg; Pkg.add("JuMP"); Pkg.add("Clp") ; Pkg.add("Cbc")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %2.9 %>                             ]  25.9 %Fetching: [================>                        ]  38.8 %                   ]  51.6 %          ]  73.8 % [===================================>     ]  86.8 %

  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


In [5]:
println("Introdução à Otimização Logística")

Introdução à Otimização Logística


Exemplo: Programação Linear - Problema da dieta

In [6]:
using JuMP, Clp

# Preparando um modelo de otimização
m = Model(Clp.Optimizer)

# Declarando as variáveis
@variable(m, x1 >= 0)
@variable(m, x2 >= 0)

# Definindo a função objetivo
@objective(m, Min, 0.6x1 + 0.35x2)

# Adicionando as restrições
@constraint(m, constraint1, 5x1 +  7x2 >= 8)
@constraint(m, constraint2,  4x1 + 2x2 >= 15)
@constraint(m, constraint3,  2x1 + x2 >= 3)

# Imprimindo o modelo criado
print(m)

# Resolvendo o modelo de otimização
status = JuMP.optimize!(m)

# Imprimindo a solução ótima
println("Solução ótima:")
println("x1 = ", JuMP.value(x1))
println("x2 = ", JuMP.value(x2))

Solução ótima:
x1 = 3.75
x2 = 0.0
Coin0506I Presolve 3 (0) rows, 2 (0) columns and 6 (0) elements
Clp0006I 0  Obj 0 Primal inf 6.3928568 (3)
Clp0006I 1  Obj 2.25
Clp0000I Optimal - objective value 2.25
Clp0032I Optimal objective 2.25 - 1 iterations time 0.002


Exemplo: Programação Linear - Problema de transporte

In [1]:
using JuMP, Clp

# Preparando um modelo de otimização
m = Model(Clp.Optimizer)

#Dados de entrada
F = [1,2]
C = [1,2,3]
dem = [8 5 2]
sup = [6 9]
custo = [5 5 3 ; 6 4 1]

# Declarando as variáveis
@variable(m, x[i in F,j in C] >= 0)

# Definindo a função objetivo
@objective(m, Min, sum(custo[i,j]*x[i,j] for i in F, j in C))

# Adicionando as restrições
@constraint(m, satisfazdemanda[j in C], sum(x[i,j] for i in F) == dem[j] )
@constraint(m, respeitasuprimentos[i in F], sum(x[i,j] for j in C) == sup[i] )

# Imprimindo o modelo criado
print(m)

# Resolvendo o modelo de otimização
JuMP.optimize!(m)

# Imprimindo a solução ótima
println("Solução ótima:")
for i in F, j in C
	println("x[",i,",",j,"] =" , JuMP.value(x[i,j]))
end

Solução ótima:
x[1,1] =6.0
x[1,2] =0.0
x[1,3] =0.0
x[2,1] =2.0
x[2,2] =5.0
x[2,3] =2.0
Coin0506I Presolve 0 (-5) rows, 0 (-6) columns and 0 (-12) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 64
Coin0511I After Postsolve, objective 64, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 64 - 0 iterations time 0.002, Presolve 0.00


Exemplo: Programação Linear Inteira - Problema da mochila inteira

In [2]:
using JuMP, Cbc

# Preparando um modelo de otimização
m = Model(Cbc.Optimizer)

#Dados de entrada

beneficios = [10, 12, 7, 5, 12, 18, 6, 4 , 3, 9]
pesos = [5, 7, 6, 10, 12, 8, 4, 2 , 3, 9]
capacidade = 18

nitens = length(beneficios)

# Declarando as variáveis
@variable(m, x[i in 1:nitens] >= 0, Int)

# Definindo a função objetivo
@objective(m, Max, sum(beneficios[i]*x[i] for i in 1:nitens))

# Adicionando as restrições
@constraint(m, restricaomochila, sum(pesos[i]*x[i] for i in 1:nitens) <= capacidade )

# Imprimindo o modelo criado
print(m)

# Resolvendo o modelo de otimização
JuMP.optimize!(m)

# Imprimindo a solução ótima
println("Solução ótima:")
for i in 1:nitens
	println("x[",i,"] =" , JuMP.value(x[i]))
end


Solução ótima:
x[1] =0.0
x[2] =0.0
x[3] =0.0
x[4] =0.0
x[5] =0.0
x[6] =2.0
x[7] =0.0
x[8] =1.0
x[9] =0.0
x[10] =0.0
Welcome to the CBC MILP Solver 
Version: 2.10.5 
Build Date: Jan  1 1970 

command line - Cbc_C_Interface -solve -quit (default strategy 1)
Continuous objective value is 40.5 - 0.00 seconds
Cgl0004I processed model has 1 rows, 10 columns (10 integer (2 of which binary)) and 10 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0012I Integer solution of -36 found by DiveCoefficient after 0 iterations and 0 nodes (0.00 seconds)
Cbc0012I Integer solution of -40 found by DiveCoefficient after 1 iterations and 0 nodes (0.00 seconds)
Cbc0031I 1 added rows had average density of 2
Cbc0013I At root node, 1 cuts changed objective from -40 to -40 in 2 passes
Cbc0014I Cut generator 0 (Probing) - 0 row cuts average 0.0 elements, 3 column cuts (3 active)  in 0.000 seconds - new frequency is 1
Cbc0014I Cut generator 1 (Gomory) - 1 row cuts average 2.0 elements, 0 column cuts (

In [ ]:
using JuMP, Cbc

# Preparando um modelo de otimização
m = Model(Cbc.Optimizer)

#Dados de entrada

beneficios = [10, 12, 7, 5, 12, 18, 6, 4 , 3, 9]
pesos = [5, 7, 6, 10, 12, 8, 4, 2 , 3, 9]
capacidade = 18

nitens = length(beneficios)

# Declarando as variáveis
@variable(m, x[i in 1:nitens] >= 0, Int)

# Definindo a função objetivo
@objective(m, Max, sum(beneficios[i]*x[i] for i in 1:nitens))

# Adicionando as restrições
@constraint(m, restricaomochila, sum(pesos[i]*x[i] for i in 1:nitens) <= capacidade )

# Imprimindo o modelo criado
print(m)

# Resolvendo o modelo de otimização
JuMP.optimize!(m)

# Imprimindo a solução ótima
println("Solução ótima:")
for i in 1:nitens
	println("x[",i,"] =" , JuMP.value(x[i]))
end
